In [ ]:
import numpy as np
import pandas as pd
import time
import pickle
from sklearn.svm import SVC
import scipy.io
import h5py
import mat73
import os


In [3]:


dev_feature = '/Users/asimadnan/Desktop/Mres/ASVspoof_2019_baseline_CM_v1/AllTrainData/CQCC_devFeatureCell_train.mat'
dev_filename = '/Users/asimadnan/Desktop/Mres/ASVspoof_2019_baseline_CM_v1/AllTrainData/CQCC_filelist_dev.mat'
dev_protocol = '/Users/asimadnan/Desktop/Mres/ASVSPOOF_DATA/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.dev.trl.txt'
model_path = '/Users/asimadnan/Desktop/Mres/Experiments/04-05-2021_04-51svm_model.svm'
output_path = '/Users/asimadnan/Desktop/Mres/Experiments/'

# dev_feature = '/scratch/fk99/ae1028/devFeatureCell_train.mat'
# dev_filename = '/scratch/fk99/ae1028/AllTrainData/CQCC_filelist_dev.mat'
# dev_protocol = '/scratch/fk99/ae1028/ASVspoof2019.LA.cm.dev.trl.txt'
# model_path = '/scratch/fk99/ae1028/models/04-05-2021_04-51svm_model.svm'
#output_path = '/scratch/fk99/ae1028/'



dev_feature_data = mat73.loadmat(dev_feature)
dev_file_names = scipy.io.loadmat(dev_filename)
labels = pd.read_csv(dev_protocol, delimiter = " ", header=None)

labels.columns = ['SPEAKER_ID','AUDIO_FILE_NAME','ENVIRONMENT_ID','ATTACK_ID','KEY']

def extract_feature_dev(data,target_num_rows,filenames):
    all_features = []
    for idx,item in enumerate(data):
        feature = np.array([np.array(xi) for xi in item])   
        if(target_num_rows - feature.shape[1]) > 0:
            feature = np.concatenate((feature, feature[:,:target_num_rows - feature.shape[1]]), axis=1)
        else:
            feature = feature[:,:target_num_rows]
        
        
        feature = feature.reshape(-1)
        feature = np.append(feature,filenames[idx][0][0])
        #print(filenames[idx][0][0])
        all_features.append(feature)
    return all_features





In [7]:
dev_data = extract_feature_dev(dev_feature_data['devFeatureCell'],160,dev_file_names['filelist'])

feature_len = len(dev_data[0]) -1

dev_data = pd.DataFrame(dev_data)
dev_data = dev_data.rename({(feature_len): 'AUDIO_FILE_NAME'}, axis=1)
dev_data = pd.merge(dev_data, labels, on='AUDIO_FILE_NAME')

X = dev_data.iloc[:,:feature_len].values
y = dev_data['KEY']



KeyboardInterrupt



In [5]:


svm_model = pickle.load(open(model_path,'rb')) 


/Users/asimadnan/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator SVC from version 0.24.2 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [6]:
svm_model.decision_function(X)


array([-1.07115735, -1.06775951, -1.18230247, -1.1548114 , -1.0520046 ,
       -0.87608851, -0.91775007, -1.08676794, -0.55227186, -0.91887556,
       -0.95399768, -1.1780256 , -1.15854029, -1.08317172, -1.11673028,
       -1.09425354, -0.99480724, -1.11189733, -1.15894957, -1.06632199,
       -1.01850427, -1.0008867 , -1.05290824, -1.0361872 , -1.1807167 ,
       -0.96798267, -0.80330872, -1.15225605, -0.73908401, -1.08332769,
       -1.23907896, -0.89741444, -0.65983199, -1.33474626, -0.93347772,
       -0.90109464, -0.94861765, -1.27346634, -1.1920126 , -0.96879658,
       -1.00043522, -1.22862348, -1.22378481, -1.14210386, -1.04016778,
       -1.01001066, -1.04588657, -1.05480593, -1.00115648, -0.89574143,
       -1.17918003, -1.24891741, -0.97417833, -0.90963967, -1.09882167,
       -0.98407153, -0.85496817, -1.15316852, -0.98481015, -1.0074647 ,
       -0.8562096 , -0.93123457, -1.13261862, -1.13435486, -0.86195161,
       -1.1091921 , -1.13390789, -0.84003058, -0.92864946, -1.25

In [11]:
y_score = svm_model.decision_function(X)

labels[['SCORE']] = y_score


labels[['AUDIO_FILE_NAME','ATTACK_ID','KEY','SCORE']].to_csv(os.path.join(output_path + 'cqcc_svm_dev_scores.csv'),index=False,sep=" ",header=False)


ValueError: cannot set using a multi-index selection indexer with a different length than the value